In [19]:
import sys
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
#import statsmodels.api as sm
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score, roc_auc_score, roc_curve
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import recall_score, classification_report
from sklearn.preprocessing import StandardScaler


In [11]:
def load_data():
    train_df= pd.read_csv('train_df.csv')
    test_df = pd.read_csv('test_df.csv')
    val_df = pd.read_csv('val_df.csv')
    return train_df, test_df, val_df

In [12]:
train_df, test_df, val_df = load_data()

In [13]:
X_train =train_df.drop('Cancer', axis=1)
y_train = train_df['Cancer']

In [14]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_train, y_train= smote.fit_resample(X_train, y_train)

In [15]:
X_test = test_df.drop('Cancer', axis = 1)
y_test = test_df['Cancer']

In [16]:
X_val = val_df.drop('Cancer', axis = 1)
y_val = val_df['Cancer']

In [17]:
# Standardize the dataset
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

In [18]:
# Define the model
def create_model(input_shape, hidden_units, activation):
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=input_shape))
    for units in hidden_units:
        model.add(layers.Dense(units, activation=activation, kernel_regularizer=tf.keras.regularizers.l2(0.01)))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

input_shape = X_train.shape[1]
hidden_units = [256]  # You can add more units or layers if needed
activation = 'relu'
model = create_model(input_shape, hidden_units, activation)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Function to evaluate the model based on recall
def eval_model(model, X_val, y_val):
    y_pred = (model.predict(X_val) > 0.48).astype("int32")
    recall = recall_score(y_val, y_pred)
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    f1= f1_score(y_val, y_pred)
    model = print(f'Recall: {recall}, Accuracy: {accuracy}, Precision: {precision}, F1_score: {f1}')
    return model

# Training and evaluating the model to choose the best one based on recall
best_recall = 0
best_model = None

for i in range(30):
    model = create_model(input_shape, hidden_units, activation)
    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    history = model.fit(X_train, y_train, epochs=5, batch_size=50, validation_data=(X_val, y_val), verbose=0)
    
    current_recall = eval_model(model, X_val, y_val)
    if current_recall > best_recall:
        best_recall = current_recall
        best_model = tf.keras.models.clone_model(model)
        best_model.set_weights(model.get_weights())

print(f'Best Validation Recall: {best_recall * 100:.2f}%')

# Evaluate the best model on the test set
eval_model(best_model, X_test, y_test)

Best Validation Recall: 74.00%


0.7453749751342749

In [21]:
# Function to evaluate the model based on recall
def eval_model(model, X_val, y_val):
    y_pred = (model.predict(X_val) > 0.48).astype("int32")
    recall = recall_score(y_val, y_pred)
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    f1= f1_score(y_val, y_pred)
    model = print(f'Recall: {recall}, Accuracy: {accuracy}, Precision: {precision}, F1_score: {f1}')
    return model
# Evaluate the best model on the test set
eval_model(best_model, X_test, y_test)

Recall: 0.7453749751342749.2f, Accuracy: 0.6483556198048428, Precision: 0.2077626836706404, F1_score: 0.32495013442025844


In [16]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import StandardScaler

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the neural network architecture
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.Recall()])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Evaluate the model
test_loss, test_recall = model.evaluate(X_test, y_test, verbose=2)
print(f'Test recall: {test_recall}')


Epoch 1/10
19623/19623 [==============================] - 21s 1ms/step - loss: 0.4991 - recall_1: 0.7916
Epoch 2/10
19623/19623 [==============================] - 21s 1ms/step - loss: 0.4414 - recall_1: 0.7946
Epoch 3/10
19623/19623 [==============================] - 29s 1ms/step - loss: 0.4199 - recall_1: 0.7957
Epoch 4/10
19623/19623 [==============================] - 31s 2ms/step - loss: 0.4061 - recall_1: 0.7955
Epoch 5/10
19623/19623 [==============================] - 28s 1ms/step - loss: 0.3983 - recall_1: 0.7961
Epoch 6/10
19623/19623 [==============================] - 34s 2ms/step - loss: 0.3924 - recall_1: 0.7961
Epoch 7/10
19623/19623 [==============================] - 34s 2ms/step - loss: 0.3881 - recall_1: 0.7959
Epoch 8/10
19623/19623 [==============================] - 34s 2ms/step - loss: 0.3850 - recall_1: 0.7972
Epoch 9/10
19623/19623 [==============================] - 34s 2ms/step - loss: 0.3821 - recall_1: 0.7979
Epoch 10/10
19623/19623 [==============================

In [ ]:
class MLP(nn.Module):
    def __init__(self,num_inputs=784,num_outputs=10,num_hiddens=256, activate = "ReLU"):
        super(MLP, self).__init__()
        ## TODO: Build your MLP network, feel free to add more arguments to the init function if needed.
        # YOUR CODE HERE
        self.activate = nn.ReLU() if activate == "ReLU" else nn.Tanh() if activate == "Tanh" else nn.Sigmoid()
        self.layer1 = nn.Linear(num_inputs, num_hiddens)
        self.layer2 = nn.Linear(num_hiddens, num_outputs)
        
        #raise NotImplementedError()

    def forward(self, X):
        ## TODO: feed data into the MLP network that you built.
    
        #raise NotImplementedError()
        X = X.view(-1, 28*28)
        X = self.activate(self.layer1(X))
        X = self.activate(self.layer2(X))
        
        
        return X
        

In [19]:
unique, counts = np.unique(y_train, return_counts=True)
print(dict(zip(unique, counts)))

{0.0: 211343, 1.0: 211343}


In [20]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the neural network architecture
model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')  
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.Recall(), 'accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64)

# Evaluate the model
test_loss, test_recall, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f'Test recall: {test_recall}')
print(f'Test accuracy: {test_accuracy}')

# Predict on test data to calculate F1 score
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Calculate F1 score
test_f1 = f1_score(y_test, y_pred)
print(f'Test F1 score: {test_f1}')

Epoch 1/10
6605/6605 [==============================] - 15s 2ms/step - loss: 0.5687 - recall_2: 0.7935 - accuracy: 0.7003
Epoch 2/10
6605/6605 [==============================] - 14s 2ms/step - loss: 0.5296 - recall_2: 0.8008 - accuracy: 0.7247
Epoch 3/10
6605/6605 [==============================] - 14s 2ms/step - loss: 0.5033 - recall_2: 0.7943 - accuracy: 0.7371
Epoch 4/10
6605/6605 [==============================] - 14s 2ms/step - loss: 0.4980 - recall_2: 0.7929 - accuracy: 0.7397
Epoch 5/10
6605/6605 [==============================] - 14s 2ms/step - loss: 0.4961 - recall_2: 0.7929 - accuracy: 0.7408
Epoch 6/10
6605/6605 [==============================] - 14s 2ms/step - loss: 0.4938 - recall_2: 0.7945 - accuracy: 0.7417
Epoch 7/10
6605/6605 [==============================] - 14s 2ms/step - loss: 0.4928 - recall_2: 0.7960 - accuracy: 0.7430
Epoch 8/10
6605/6605 [==============================] - 14s 2ms/step - loss: 0.4907 - recall_2: 0.7965 - accuracy: 0.7442
Epoch 9/10
6605/6605 [==

In [22]:
df = pd.read_csv("clean_df (2).csv")

In [24]:
X= df.drop('Cancer', axis=1)
y=df.Cancer

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [25]:
imputer = IterativeImputer(max_iter = 3, random_state=0)
X_train = imputer.fit_transform(X_train)

/home/rdemoz/.local/lib/python3.9/site-packages/sklearn/impute/_iterative.py:825: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [26]:
imputer = IterativeImputer(max_iter = 3, random_state=0)
X_test = imputer.fit_transform(X_test)

/home/rdemoz/.local/lib/python3.9/site-packages/sklearn/impute/_iterative.py:825: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [35]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_train, y_train= smote.fit_resample(X_train, y_train)

In [37]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import layers, models
def build_model(optimizer='adam'):
    model = models.Sequential([
        layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=[tf.keras.metrics.Recall()])
    return model

keras_clf = KerasClassifier(build_fn=build_model, verbose=0)

/tmp/ipykernel_1545226/3453302362.py:14: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  keras_clf = KerasClassifier(build_fn=build_model, verbose=0)
